## Set up directory

In [1]:
%cd /Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect


/Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect


# Example using custom datahelper and Faster-RCNN

In [2]:
# import packages
from datahelper import CreateDataset
import torch
from torch.utils.data import DataLoader
from PIL import Image
import os
import torchvision
#from model import MVP
from tqdm import tqdm
import os
import pandas as pd
import pickle

In [3]:
# Create paths
pickle_path = "/Users/ianleefmans/Desktop/Insight/Project/Re-Identifying_Persistent_Skin_Conditions/skinConditionDetect/annotation_dict.pkl"
data_directory = "/Users/ianleefmans/Desktop/Insight/Project/Data"

# Create dataset
dataset = CreateDataset(pickle_path, data_directory, transform = torchvision.transforms.ToTensor())
def my_collate(batch):
    data = [item[0] for item in batch]
    target = [item[1] for item in batch]
    #target = torch.LongTensor(target)
    return data, target
train_loader = DataLoader(dataset=dataset, batch_size=4, num_workers=4, shuffle=True, collate_fn=my_collate)
sample = iter(train_loader).next()


In [4]:
lis =[]
for i in tqdm(range(len(dataset))):
    try:
        dataset[i]
        if dataset[i]
    except:
        lis.append(i)
        print(i)
    finally:
        pass
lis

  0%|          | 976/232800 [01:37<4:42:24, 13.68it/s] 

973


  1%|          | 2323/232800 [03:35<6:56:33,  9.22it/s] 

2321


  1%|▏         | 3051/232800 [04:35<5:39:26, 11.28it/s]

...

232657


100%|█████████▉| 232686/232800 [5:48:30<00:07, 14.72it/s]

232684


100%|██████████| 232800/232800 [5:48:39<00:00, 11.13it/s]


[973,
 2321,
 3049,
 5296,
 6021,
 7757,
 7797,
 8278,
 8279,
 8757,
 9153,
 9191,
 11417,
 11452,
 12006,
 12768,
 13499,
 13524,
 13693,
 15949,
 17199,
 17219,
 17614,
 17615,
 17794,
 18324,
 18999,
 19475,
 20597,
 20653,
 21940,
 23628,
 23650,
 23738,
 23906,
 24057,
 25922,
 26038,
 26053,
 26068,
 28268,
 28484,
 30338,
 31109,
 33298,
 33300,
 33301,
 33302,
 33676,
 33989,
 34994,
 35425,
 35997,
 35998,
 36459,
 37196,
 37497,
 37499,
 37501,
 37849,
 37855,
 38251,
 38687,
 38852,
 38863,
 38870,
 38884,
 38968,
 40250,
 40251,
 40252,
 40494,
 42016,
 42028,
 42046,
 42064,
 42696,
 44178,
 44180,
 45176,
 45180,
 47030,
 47048,
 47073,
 47287,
 48242,
 49147,
 49390,
 49516,
 51766,
 51888,
 53901,
 53928,
 53976,
 54384,
 54386,
 54387,
 55995,
 58319,
 59405,
 59407,
 59742,
 59812,
 60883,
 61395,
 61396,
 61669,
 61854,
 61965,
 62703,
 62710,
 62942,
 63265,
 65204,
 65642,
 65649,
 65650,
 65834,
 66446,
 66493,
 67064,
 67079,
 67339,
 67344,
 67985,
 68001,
 6831

In [16]:
len(lis)

478

In [ ]:
image = sample[0]
annotation = sample[1]

In [3]:
# Instantiate Minimum Viable Product (Faster-RCNN)
mvp = MVP()
# Train on example image and annotation
output = mvp(image, annotation)

NameError: name 'MVP' is not defined

In [21]:
output

{'loss_classifier': tensor(1.9176, grad_fn=<NllLossBackward>),
 'loss_box_reg': tensor(0.0086, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(9.6734, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 'loss_rpn_box_reg': tensor(10.4392, grad_fn=<DivBackward0>)}

In [36]:
import pandas as pd
data_path = "/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/"

test_json = 'user_000000000277.json'

df = pd.read_json(data_path+test_json)



In [ ]:
mvp.eval()
mvp


In [95]:
from tqdm import tqdm
import os
import pandas as pd
import pickle
annotation_dict = {}
count = 0 
for filename in tqdm(os.listdir("/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/")):
    try:
        df = pd.read_json("/Users/ianleefmans/Desktop/Insight/Project/Data/followup_data/"+filename)
        df = df.loc[(df.image_details.notnull()) & (df.image_path.notnull()),:]
        for i in list(df.index.values):
            try:
                df.iloc[i].image_details['annotation'] !=None
                df.iloc[i]['image_details']['annotation'][0]
                ok = False
                for j in df.iloc[i].image_details['annotation']:
                    if (j['condition']=='Detected') and ('bounding_boxes' in j.keys()):
                        if len(j['bounding_boxes'])>0:
                            ok = True
                if ok ==False:
                    a = 2/0
                    
                annotation_dict[count] = (filename, i)
                count+=1
            except:
                print
            finally:
                pass
            
    except:
        pass
    finally:
        pass
with open("annotation_dict.pkl", 'wb') as handle:
    pickle.dump(annotation_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 17293/17293 [08:10<00:00, 35.29it/s]


In [8]:
# open pickl3
with open("annotation_dict.pkl", 'rb') as handle:
    dictionary = pickle.load(handle)


In [12]:
lis[:12]

[973, 2321, 3049, 5296, 6021, 7757, 7797, 8278, 8279, 8757, 9153, 9191]

In [12]:
#remove bad jsons first round out of 10000
bad_json = [2321, 3049, 5296, 6021, 7757, 7797, 8278, 8279, 8757, 9153, 9191]
for i in bad_json:
    dictionary.pop(i)

In [15]:
#remove bad jsons second round out of all
bad_json = lis
for i in bad_json[12:]:
    dictionary.pop(i)

In [18]:
# save pickle
with open("annotation_dict.pkl", 'wb') as handle:
    pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
len(dictionary) # dictionary length after removing 467 bad json

232333

In [9]:
len(dictionary) # dictionary length after removing 11 bad json

232800

In [79]:
len(dictionary) # dictionary length before removing 11 bad json


232811